In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
import numpy as np
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold
from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.optimizers import RMSprop
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.regularizers import l1_l2

In [2]:
# Database connection variables
username = 'postgres'
password = 'postgres'
database_name = 'housing_data_db'
host = 'localhost' # or your database host
port = '5432'

# Create the connection string for SQLAlchemy
connection_string = f'postgresql://{username}:{password}@{host}:{port}/{database_name}'

# Create the database engine
engine = create_engine(connection_string)

# Query to select all data
query = 'SELECT * FROM housing'

# Fetch data into DataFrame
df = pd.read_sql(query, engine)

df['mainroad'] = df['mainroad'].astype(int)
df['guestroom'] = df['guestroom'].astype(int)
df['basement'] = df['basement'].astype(int)
df['hotwaterheating'] = df['hotwaterheating'].astype(int)
df['airconditioning'] = df['airconditioning'].astype(int)
df['prefarea'] = df['prefarea'].astype(int)

df['rooms'] = df['bathrooms'] + df['bedrooms'] + df['guestroom'] 
df['area_per_room'] = df['area'] / df['rooms']

df.head()

correlation_matrix = df.corr()
features_to_use = correlation_matrix['price'].sort_values(key=abs, ascending=False).index[1:]  # top correlated features

In [3]:
# Convert boolean columns to integers
boolean_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
for col in boolean_columns:
    df[col] = df[col].astype(int)

In [4]:
top_features = ['area', 'rooms', 'stories', 'parking', 'area_log','parking','area_per_room']  # Based on correlation matrix
X = df[top_features]
y = df['price']

# Add polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [8]:
model = Sequential()
model.add(Dense(20, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))



# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

In [21]:
early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# Train the model
history = model.fit(
    X_train_scaled, 
    y_train, 
    epochs=150, 
    batch_size=10, 
    validation_split=0.1, 
    callbacks=[early_stopping]
)

Epoch 1/150
73/73 [==============================] - 0s 709us/step - loss: 0.2407 - val_loss: 0.4152
Epoch 2/150
73/73 [==============================] - 0s 411us/step - loss: 0.2402 - val_loss: 0.3986
Epoch 3/150
73/73 [==============================] - 0s 400us/step - loss: 0.2487 - val_loss: 0.3963
Epoch 4/150
73/73 [==============================] - 0s 395us/step - loss: 0.2434 - val_loss: 0.4185
Epoch 5/150
73/73 [==============================] - 0s 389us/step - loss: 0.2430 - val_loss: 0.3984
Epoch 6/150
73/73 [==============================] - 0s 398us/step - loss: 0.2403 - val_loss: 0.4041
Epoch 7/150
73/73 [==============================] - 0s 394us/step - loss: 0.2453 - val_loss: 0.4011
Epoch 8/150
73/73 [==============================] - 0s 392us/step - loss: 0.2399 - val_loss: 0.4059
Epoch 9/150
73/73 [==============================] - 0s 383us/step - loss: 0.2430 - val_loss: 0.4006
Epoch 10/150
73/73 [==============================] - 0s 404us/step - loss: 0.2424 - val_lo

73/73 [==============================] - 0s 396us/step - loss: 0.2385 - val_loss: 0.3991
Epoch 82/150
73/73 [==============================] - 0s 389us/step - loss: 0.2368 - val_loss: 0.4141
Epoch 83/150
73/73 [==============================] - 0s 392us/step - loss: 0.2386 - val_loss: 0.4132
Epoch 84/150
73/73 [==============================] - 0s 385us/step - loss: 0.2349 - val_loss: 0.4090
Epoch 85/150
73/73 [==============================] - 0s 401us/step - loss: 0.2371 - val_loss: 0.4117
Epoch 86/150
73/73 [==============================] - 0s 403us/step - loss: 0.2370 - val_loss: 0.3981
Epoch 87/150
73/73 [==============================] - 0s 395us/step - loss: 0.2413 - val_loss: 0.4087
Epoch 88/150
73/73 [==============================] - 0s 385us/step - loss: 0.2394 - val_loss: 0.4073
Epoch 89/150
73/73 [==============================] - 0s 399us/step - loss: 0.2393 - val_loss: 0.4136
Epoch 90/150
73/73 [==============================] - 0s 396us/step - loss: 0.2371 - val_loss: 

In [22]:
# Predict and evaluate
y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'R-squared: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')

7/7 [==============================] - 0s 528us/step
R-squared: -0.28
RMSE: 0.63
